[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/explainable_retrieval.ipynb)

# 文档搜索中的可解释检索

## 概述

此代码实现了一个可解释的检索器，该系统不仅能根据查询检索相关文档，还能为每个检索到的文档提供相关性解释。它将基于向量的相似性搜索与自然语言解释相结合，增强了检索过程的透明度和可解释性。

## 动机

传统的文档检索系统通常像黑匣子一样工作，提供结果却不解释选择原因。在理解结果背后的推理至关重要的场景中，这种缺乏透明度可能会带来问题。可解释检索器通过提供对每个检索文档相关性的见解来解决此问题。

## 关键组件

1. 从输入文本创建向量存储
2. 使用 FAISS 的基础检索器进行高效的相似性搜索
3. 用于生成解释的语言模型 (LLM)
4. 结合检索和解释生成的自定义 ExplainableRetriever 类

## 方法详情

### 文档预处理和向量存储创建

1. 使用 OpenAI 的嵌入模型将输入文本转换为嵌入。
2. 从这些嵌入创建一个 FAISS 向量存储，以进行高效的相似性搜索。

### 检索器设置

1. 从向量存储创建一个基础检索器，配置为返回前 5 个最相似的文档。

### 解释生成

1. 使用 LLM (GPT-4) 生成解释。
2. 定义一个自定义提示模板，以指导 LLM 解释检索文档的相关性。

### ExplainableRetriever 类

1. 将基础检索器和解释生成结合到一个单一接口中。
2. `retrieve_and_explain` 方法：
   - 使用基础检索器检索相关文档。
   - 为每个检索到的文档生成其与查询相关性的解释。
   - 返回一个包含文档内容及其解释的字典列表。

## 此方法的优点

1. 透明度：用户可以理解为什么检索到特定的文档。
2. 信任：解释能建立用户对系统结果的信心。
3. 学习：用户可以深入了解查询和文档之间的关系。
4. 调试：更容易识别和纠正检索过程中的问题。
5. 定制化：可以为不同的用例或领域定制解释提示。

## 结论

可解释检索器是向更具可解释性和可信赖性的信息检索系统迈出的重要一步。通过在检索到的文档旁边提供自然语言解释，它弥合了强大的基于向量的搜索技术与人类理解之间的差距。这种方法在信息检索背后的推理与检索到的信息本身同等重要的各个领域具有潜在应用，例如法律研究、医疗信息系统和教育工具。

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [ ]:
# Install required packages
!pip install python-dotenv

In [ ]:
# Clone the repository to access helper functions and evaluation modules
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# If you need to run with the latest data
# !cp -r RAG_TECHNIQUES/data .

In [ ]:
import os
import sys
from dotenv import load_dotenv


# Original path append replaced for Colab compatibility
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define the explainable retriever class 

In [6]:
class ExplainableRetriever:
    def __init__(self, texts):
        self.embeddings = OpenAIEmbeddings()

        self.vectorstore = FAISS.from_texts(texts, self.embeddings)
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)

        
        # Create a base retriever
        self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": 5})
        
        # Create an explanation chain
        explain_prompt = PromptTemplate(
            input_variables=["query", "context"],
            template="""
            Analyze the relationship between the following query and the retrieved context.
            Explain why this context is relevant to the query and how it might help answer the query.
            
            Query: {query}
            
            Context: {context}
            
            Explanation:
            """
        )
        self.explain_chain = explain_prompt | self.llm

    def retrieve_and_explain(self, query):
        # Retrieve relevant documents
        docs = self.retriever.get_relevant_documents(query)
        
        explained_results = []
        
        for doc in docs:
            # Generate explanation
            input_data = {"query": query, "context": doc.page_content}
            explanation = self.explain_chain.invoke(input_data).content
            
            explained_results.append({
                "content": doc.page_content,
                "explanation": explanation
            })
        
        return explained_results



### Create a mock example and explainable retriever instance

In [7]:

# Usage
texts = [
    "The sky is blue because of the way sunlight interacts with the atmosphere.",
    "Photosynthesis is the process by which plants use sunlight to produce energy.",
    "Global warming is caused by the increase of greenhouse gases in Earth's atmosphere."
]

explainable_retriever = ExplainableRetriever(texts)


### 展示结果

In [ ]:
query = "Why is the sky blue?"
results = explainable_retriever.retrieve_and_explain(query)

for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(f"Content: {result['content']}")
    print(f"Explanation: {result['explanation']}")
    print()